# 抓取 Googleplay 的 APP 評論與評分
1. 記得先檢查 Chrome 的版本
2. 搜尋 Chrome Driver ，並下載對應的驅動程式
3. 將檔案解壓縮至 Anaconda / miniconda 的路徑

## 安裝套件

In [1]:
!pip install selenium
!pip install bs4
!pip install pandas
!pip install openpyxl

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


## 載入套件

In [25]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
import time

## 打開想爬的網頁

- Foodpanda 的APP頁面作為範例
  - https://play.google.com/store/apps/details?id=com.global.foodpanda.android&hl=zh_TW&showAllReviews=true

- Ubereats
  - https://play.google.com/store/apps/details?id=com.ubercab.eats&hl=zh_TW&showAllReviews=true

In [26]:
driver = webdriver.Chrome()
driver.get('https://play.google.com/store/apps/details?id=com.global.foodpanda.android&hl=zh_TW&showAllReviews=true')
time.sleep(1)
driver.find_element_by_xpath('//div[@class="MocG8c UFSXYb LMgvRb KKjvXb"]').click()
time.sleep(1)
try:
    driver.find_element_by_xpath('//div[@jsname="V68bde"]//span[@class="vRMGwf oJeWuf"]').click()
except:
    pass

## 加載更多留言
- 透過觀察的方式可以發現，當網頁滾動到最底部的時候會自動加載更多留言，但是每加載5次會需要點擊一次「顯示更多內容」
- 另外需要留意第一次與第二次之後的「顯示更多內容」是不同的按鈕，所以在這裡使用了一個 try 函數
- 在這裡我設定做 10個點擊「顯示更多內容」，實際上可以依據資料量的大小自行增加或減少

In [27]:
for i in range(20):
    print(i)
    for j in range(5):
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        driver.find_element_by_xpath('//div[@class="U26fgb O0WRkf oG5Srb C0oVfc n9lfJ M9Bg4d"]').click()
    except:
        try:
            driver.find_element_by_xpath('//div[@class="U26fgb O0WRkf oG5Srb C0oVfc n9lfJ"]').click()
        except:
            pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


## 檢視完整留言
- 對於長度較長的留言需要另外點擊「完整留言」的按鈕才會顯示完整留言，顯示後我們才能抓到資料!
- 這裡我找的是頁面中有沒有「完整留言」的按鈕可以點擊，有的話就點擊

In [28]:
SeeMore = '//button[@jsname="gxjVle"]'
driver.execute_script("window.scrollTo(0, 0);")
for i, element in enumerate(driver.find_elements_by_xpath(SeeMore)):
    if i % 10 == 0:
        print(i)
    element.click()

0
10
20
30


## 解析資料
- 當我們把所有資料加載完成後，我們的資料是html的半結構化的網頁資料
- 因此我們在這裡需要透過BeautifulSoup來幫我們把資料轉成DataFrame

In [29]:
soup = BeautifulSoup(driver.page_source)
# driver.close()

In [30]:
list_of_comments = []
for i, element in enumerate(soup.find_all('div',{'jscontroller':'H6eOGe'})):
    if i % 10 == 0:
        print(i)
    try:
        likecounts = element.find('div',{'class':'jUL89d y92BAb'}).text
    except:
        likecounts = str(0)
    
    Content = element.find('span',{'jsname':'bN97Pc'}).text
    # 如果「完整評論」裡面有內容就抓裡面的內容
    if len(element.find('span',{'jsname':'fbQN7e'}).text) >=20:
        Content = element.find('span',{'jsname':'fbQN7e'}).text
    ndf = pd.DataFrame([{'Name':element.find('span', {'class':'X43Kjb'}).text,
                         'Sentimen':element.find('div', {'class':'pf5lIe'}).find('div')['aria-label'],
                         'likecounts':likecounts,
                         'Timestamp':element.find('span',{'class':'p2TkOb'}).text,
                         'Content':Content}],
                        columns = ['Name', 'Timestamp', 'Content', 'Sentimen', 'likecounts'])
    list_of_comments.append(ndf)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [31]:
comments = pd.concat(list_of_comments, ignore_index=True)
print(len(comments))
comments.head(20)

3120


,Name,Timestamp,Content,Sentimen,likecounts
0,萌泰泰大發,2020年6月29日,不知道怎麼那麼多人都遇到不好的經驗，都給1顆星，我用了好幾個月覺得非常好，客服在線上也回覆很...,獲評為 5 顆星 (滿分 5 顆星),0
1,Tien Elaine,2020年6月29日,差，點原味飲料，無瓶裝，未告知客戶，店家擅改成杯裝,獲評為 1 顆星 (滿分 5 顆星),0
2,黃政琦,2020年6月29日,贊👍,獲評為 5 顆星 (滿分 5 顆星),0
3,呂唯鈴,2020年6月29日,好用♥,獲評為 4 顆星 (滿分 5 顆星),0
4,Google 使用者,2020年6月29日,不錯,獲評為 3 顆星 (滿分 5 顆星),0
5,曾子桓,2020年6月29日,買麥當勞的薯條，每次來都是半盒，請問喔，你們的人員是都沒有吃飯嗎？上次還看到他再吃我的薯條跟...,獲評為 3 顆星 (滿分 5 顆星),0
6,許箏,2020年6月29日,讚,獲評為 5 顆星 (滿分 5 顆星),0
7,李宗翰,2020年6月29日,不錯,獲評為 5 顆星 (滿分 5 顆星),0
8,Ning Hsu,2020年6月29日,已經等了有一週了，還沒收到退費的簡訊通知，若我到銀行確認並無退費，那我會尋求法律幫助。（20...,獲評為 1 顆星 (滿分 5 顆星),13
9,宋佳宸,2020年6月29日,還不錯，挺方便的,獲評為 5 顆星 (滿分 5 顆星),0


In [24]:
import os
comments.to_excel('./GooglePlay_Comments.xlsx')